# House Price Dataset - Preprocessing

This section involves the preprocessing of the House Price dataset, which includes various features related to residential properties and their sale prices. The goal of preprocessing is to clean, transform, and prepare the data for machine learning models to accurately predict house prices.
**Steps Covered:**

1. Load the dataset and perform basic inspection.

2. Handle missing values using appropriate imputation or removal techniques.

3. Convert data types to suitable formats (e.g., categorical, datetime).

4. Detect and treat outliers using statistical methods.

5. Encode categorical variables using label or one-hot encoding.

6. Clean and standardise data entries for consistency.

7. Scale numerical features using normalisation or standardisation.

8. Perform basic feature engineering to create helpful derived features.


In [1]:
#imports

import pandas as pd

**Step-1:** Import the dataset

In [2]:
df = pd.read_csv("/kaggle/input/housepricedataset/house_price_train.csv")
print(df.shape)
df.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


**seperate the target variable**

In [3]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

**Step-2:** Clean and standardise data entries for consistency

In [4]:
#Strip leading/trailing whitespace from string columns
for col in X.select_dtypes(include=['object', 'category']).columns:
    X[col] = X[col].astype(str).str.strip()  # convert to string first to avoid errors

#  Convert text columns to lowercase for consistency
for col in X.select_dtypes(include=['object', 'category']).columns:
    X[col] = X[col].str.lower()

#  Replace common inconsistent entries (example)
# You can customize this based on your dataset
replace_dict = {
    'na': None,
    'n/a': None,
    'none': None,
    '?': None
}
for col in X.select_dtypes(include=['object', 'category']).columns:
    X[col] = X[col].replace(replace_dict)

# Remove duplicate rows (if necessary)
X = X.drop_duplicates()

# 5️Optional: Reset index after dropping duplicates
X = X.reset_index(drop=True)


**Step-3:** Handling the Null values in the dataset

In [5]:
X.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

In [6]:
#Drop the null values if you have more than 30% of Null

X.dropna(axis=1, thresh=0.7*len(X), inplace=True)

#Fill the null values
# Separate numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

X[numerical_cols] = X[numerical_cols].fillna(X[numerical_cols].median())

for col in categorical_cols:
    X[col] = X[col].fillna(X[col].mode()[0])


In [7]:
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,rl,65.0,8450,pave,nan,reg,lvl,allpub,...,0,0,nan,nan,nan,0,2,2008,wd,normal
1,2,20,rl,80.0,9600,pave,nan,reg,lvl,allpub,...,0,0,nan,nan,nan,0,5,2007,wd,normal
2,3,60,rl,68.0,11250,pave,nan,ir1,lvl,allpub,...,0,0,nan,nan,nan,0,9,2008,wd,normal
3,4,70,rl,60.0,9550,pave,nan,ir1,lvl,allpub,...,0,0,nan,nan,nan,0,2,2006,wd,abnorml
4,5,60,rl,84.0,14260,pave,nan,ir1,lvl,allpub,...,0,0,nan,nan,nan,0,12,2008,wd,normal


**Step-4:** Convert columns to appropriate types (i.e object --> category)

In [8]:
print(X.dtypes)

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 80, dtype: object


In [9]:
import pandas as pd

# Convert 'object' columns with few unique values to 'category'
for col in X.select_dtypes(include='object').columns:
    if X[col].nunique() < 20:  # adjust threshold based on domain knowledge
        X[col] = X[col].astype('category')

# Convert string columns that look like dates to datetime (if any)
date_cols = ['YrSold', 'MoSold']  # Replace with your actual datetime-like columns if any

for col in date_cols:
    if col in X.columns:
        try:
            X[col] = pd.to_datetime(X[col])
        except Exception:
            pass  # Keep original if conversion fails

# Fix numeric columns stored as strings (if any)
for col in X.select_dtypes(include='object').columns:
    try:
        X[col] = pd.to_numeric(X[col])
    except ValueError:
        pass  # Keep original if truly non-numeric


**Step-5:** Outlier Detection and Treatment

In [10]:
import pandas as pd

# Identify numerical columns only
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Outlier treatment using IQR method
for col in numeric_cols:
    Q1 = X[col].quantile(0.25)
    Q3 = X[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Print number of outliers
    outliers = X[(X[col] < lower_bound) | (X[col] > upper_bound)]
    print(f"{col}: {len(outliers)} outliers")

    # Optional: Remove outliers
    # X = X[(X[col] >= lower_bound) & (X[col] <= upper_bound)]

    # Optional: Cap outliers instead of removing
    # X[col] = X[col].clip(lower=lower_bound, upper=upper_bound)


Id: 0 outliers
MSSubClass: 103 outliers
LotFrontage: 106 outliers
LotArea: 69 outliers
OverallQual: 2 outliers
OverallCond: 125 outliers
YearBuilt: 7 outliers
YearRemodAdd: 0 outliers
MasVnrArea: 98 outliers
BsmtFinSF1: 7 outliers
BsmtFinSF2: 167 outliers
BsmtUnfSF: 29 outliers
TotalBsmtSF: 61 outliers
1stFlrSF: 20 outliers
2ndFlrSF: 2 outliers
LowQualFinSF: 26 outliers
GrLivArea: 31 outliers
BsmtFullBath: 1 outliers
BsmtHalfBath: 82 outliers
FullBath: 0 outliers
HalfBath: 0 outliers
BedroomAbvGr: 35 outliers
KitchenAbvGr: 68 outliers
TotRmsAbvGrd: 30 outliers
Fireplaces: 5 outliers
GarageYrBlt: 1 outliers
GarageCars: 5 outliers
GarageArea: 21 outliers
WoodDeckSF: 32 outliers
OpenPorchSF: 77 outliers
EnclosedPorch: 208 outliers
3SsnPorch: 24 outliers
ScreenPorch: 116 outliers
PoolArea: 7 outliers
MiscVal: 52 outliers


**Step-6:** Encode the categorical variables

In [11]:
import pandas as pd

# Identify categorical columns
cat_cols = X.select_dtypes(include=['object', 'category']).columns

# Apply One-Hot Encoding to all categorical columns
X_encoded = pd.get_dummies(X, columns=cat_cols, drop_first=True)

# Now X_encoded is ready for feature engineering or modeling


In [12]:
X_encoded.head

<bound method NDFrame.head of         Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
0        1          60         65.0     8450            7            5   
1        2          20         80.0     9600            6            8   
2        3          60         68.0    11250            7            5   
3        4          70         60.0     9550            7            5   
4        5          60         84.0    14260            8            5   
...    ...         ...          ...      ...          ...          ...   
1455  1456          60         62.0     7917            6            5   
1456  1457          20         85.0    13175            6            6   
1457  1458          70         66.0     9042            7            9   
1458  1459          20         68.0     9717            5            6   
1459  1460          20         75.0     9937            5            6   

      YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  ...  SaleType_conlw  \
0  